## Imports


In [1]:
import os
import pandas as pd
import numpy as np
import random
import math
from networkx.readwrite import json_graph
import json
import networkx as nx
import pickle
import logging
import logging.config
from opencensus.ext.azure.log_exporter import AzureLogHandler
import datetime

from flask import Flask,request,Response
from flask_cors import CORS
from opencensus.ext.azure.trace_exporter import AzureExporter
from opencensus.ext.flask.flask_middleware import FlaskMiddleware
from opencensus.trace.samplers import ProbabilitySampler

## Global variables

In [2]:
df_intra = pd.DataFrame()
df_intra_trim = pd.DataFrame()
df_extra = pd.DataFrame()
df_extra_trim = pd.DataFrame()

## Parameters

In [3]:
FILE_SEP = ","
PROD_DIGITS = 3  # number of digits to classify transports
MAX_NODES = 70
CHUNCK_SIZE = 5

# COMEXT DATASETS
INTRA_FILE = "data" + os.sep + "cpa_intra.csv"
EXTRA_FILE = "data" + os.sep + "tr_extra_ue.csv"
INTRA_TRIM_FILE = "data" + os.sep + "cpa_trim.csv"
EXTRA_TRIM_FILE = "data" + os.sep + "tr_extra_ue_trim.csv"

CRITERION = "VALUE_IN_EUROS"  # VALUE_IN_EUROS QUANTITY_IN_KG

In [4]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logger = logging.getLogger(__name__)

## Azure setup


In [5]:
def is_application_insight_configured():
    return (
        os.getenv("APPINSIGHTS_INSTRUMENTATIONKEY") != None
        or os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING") != None
    )

def ai_callback_function(envelope):
    if os.getenv("CLOUD_ROLE") != None:
        envelope.tags["ai.cloud.role"] = os.getenv("CLOUD_ROLE")


if is_application_insight_configured():
    log_handler = AzureLogHandler()
    log_handler.add_telemetry_processor(ai_callback_function)
    logger.addHandler(log_handler)
else:
    logger.warning("Application insights is not configured.")

2024-01-03 12:51:51.146 WARNING 3646994727 - <module>: Application insights is not configured.


## Data loading functions


In [6]:
def load_intra():
    logger.info("[TERRA] Loading dataset INTRA...")

    df = pd.read_csv(
        INTRA_FILE,
        low_memory=False,
        dtype={
            "PRODUCT": object,
            "FLOW": np.int8,
            "PERIOD": np.int32,
            "TRANSPORT_MODE": np.int8,
        },
    )

    logger.info("[TERRA] Dataset INTRA loaded!")
    logger.info(f"[TERRA] Dataset INTRA contains {df.shape[0]} records")
    return df

def load_intra_trim():
    def funcTrim(x):
        return np.int32(x.replace("T", "0"))

    logger.info("[TERRA] Loading dataset INTRA TRIMESTER...")

    df = pd.read_csv(
        INTRA_TRIM_FILE,
        low_memory=False,
        converters={"trimestre": funcTrim},
        dtype={"cpa": object, "FLOW": np.int8},
    )
    df = df[["DECLARANT_ISO", "PARTNER_ISO", "FLOW", "cpa", "trimestre", "val_cpa"]]
    df.columns = [
        "DECLARANT_ISO",
        "PARTNER_ISO",
        "FLOW",
        "PRODUCT",
        "PERIOD",
        "VALUE_IN_EUROS",
    ]
    df = df[df.PRODUCT.apply(lambda x: len(x.strip()) == PROD_DIGITS)]

    logger.info("[TERRA] Dataset INTRA TRIMESTER loaded!")
    logger.info(f"[TERRA] Dataset INTRA TRIMESTER contains {df.shape[0]} records")
    return df

def load_extra():
    logger.info("[TERRA] Loading dataset EXTRA...")

    df = pd.read_csv(
        EXTRA_FILE,
        sep=FILE_SEP,
        dtype={
            "PRODUCT_NSTR": object,
            "FLOW": np.int8,
            "PERIOD": np.int32,
            "TRANSPORT_MODE": np.int8,
        },
    )
    df.columns = [
        "PRODUCT",
        "DECLARANT_ISO",
        "PARTNER_ISO",
        "PERIOD",
        "TRANSPORT_MODE",
        "FLOW",
        "VALUE_IN_EUROS",
        "QUANTITY_IN_KG",
    ]

    logger.info("[TERRA] Dataset EXTRA loaded!")
    logger.info(f"[TERRA] Dataset EXTRA contains {df.shape[0]} records")
    return df


def load_extra_trim():
    def funcTrim(x):
        return np.int32(x.replace("T", "0"))

    logger.info("[TERRA] Loading dataset EXTRA TRIM...")

    df = pd.read_csv(
        EXTRA_TRIM_FILE,
        low_memory=False,
        converters={"TRIMESTRE": funcTrim},
        dtype={"PRODUCT_NSTR": object, "FLOW": np.int8},
    )
    df = df[
        [
            "DECLARANT_ISO",
            "PARTNER_ISO",
            "FLOW",
            "PRODUCT_NSTR",
            "TRANSPORT_MODE",
            "TRIMESTRE",
            "VALUE_IN_EUROS",
        ]
    ]
    df.columns = [
        "DECLARANT_ISO",
        "PARTNER_ISO",
        "FLOW",
        "PRODUCT",
        "TRANSPORT_MODE",
        "PERIOD",
        "VALUE_IN_EUROS",
    ]
    df = df[df.PRODUCT.apply(lambda x: len(x.strip()) == PROD_DIGITS)]

    logger.info("[TERRA] Dataset EXTRA TRIM loaded!")
    logger.info(f"[TERRA] Dataset EXTRA TRIM contains {df.shape[0]} records")
    return df


## Utility functions

In [7]:
# Build a query to delete edges in the graph
def build_edges_query(edges, flow):
    
    # Empty query object
    query = []
    
    for edge in edges:
        if flow == 1:
            PARTNER_ISO = edge["from"]
            DECLARANT_ISO = edge["to"]
        else:
            DECLARANT_ISO = edge["from"]
            PARTNER_ISO = edge["to"]

        exclude = str(edge["exclude"])

        # Graph without TRANSPORTS
        if "-99" in exclude:
            query.append(
                "(DECLARANT_ISO == '"
                + DECLARANT_ISO
                + "' & PARTNER_ISO == '"
                + PARTNER_ISO
                + "' )"
            )
        else:
            query.append(
                "((DECLARANT_ISO == '"
                + DECLARANT_ISO
                + "' & PARTNER_ISO == '"
                + PARTNER_ISO
                + "' & TRANSPORT_MODE in "
                + exclude
                + "))"
            )
    return "not (" + ("|".join(query)) + ")"


# Remove from the transport dataframe the subset NOT containing edges
def remove_edges(df_comext, edges, flow):
    query = build_edges_query(edges, flow)
    df_comext = df_comext.query(query)
    return df_comext


def build_metrics(graph):
    logger.info("[TERRA] Calculating graph metrics...")

    in_deg = nx.in_degree_centrality(graph)
    graph_metrics = {}
    vulnerability = {}

    for k, v in in_deg.items():
        if v != 0:
            vulnerability[k] = 1 - v
        else:
            vulnerability[k] = 0
        
        graph_metrics = {
            "degree_centrality": nx.degree_centrality(graph),
            "density": nx.density(graph),
            "vulnerability": vulnerability,
            "exportation strenght": {
                a: b for a, b in graph.out_degree(weight="weight")
            },
            "hubness": nx.closeness_centrality(graph.to_undirected()),
        }
    
    logger.info("[TERRA] Graph metrics ready!")
    return graph_metrics

## Data processing

### Extract graph table

In [8]:
def extract_graph_table(
    period,
    percentage,
    transports,
    flow,
    product,
    criterion,
    selectedEdges,
    df_comext,
):

    logger.info("[TERRA] Preparing graph table...")

    # Extract FLOW
    df_comext = df_comext[
        df_comext["FLOW"] == flow
    ]
    
    # Extract PERIOD
    if period is not None:
        period = np.int32(period)
        df_comext = df_comext[
            df_comext["PERIOD"] == period
        ]
    
    # Extract TRANSPORTS
    if transports is not None:
        df_comext = df_comext[
            df_comext["TRANSPORT_MODE"].isin(transports)
        ]

    # Extract PRODUCT
    if product is not None:
        df_comext = df_comext[
            df_comext["PRODUCT"] == product
        ]

    # Extract EDGES
    if selectedEdges is not None:
        
        NUMBER_OF_EDGES_CHUNKS = len(selectedEdges) // (CHUNCK_SIZE)

        for i in range(NUMBER_OF_EDGES_CHUNKS):
            selectedEdges_i = selectedEdges[
                i * CHUNCK_SIZE : (i + 1) * CHUNCK_SIZE
            ]
            df_comext = remove_edges(
                df_comext, selectedEdges_i, flow
            )
        selectedEdges_i = selectedEdges[
            NUMBER_OF_EDGES_CHUNKS * CHUNCK_SIZE : len(selectedEdges)
        ]
        
        if len(selectedEdges_i) > 0:
            df_comext = remove_edges(
                df_comext, selectedEdges_i, flow
            )

    # Aggregate on DECLARANT_ISO and PARTNER_ISO and sort on criterion (VALUE or QUANTITY)
    df_comext = (
        df_comext.groupby(["DECLARANT_ISO", "PARTNER_ISO"])
        .sum()
        .reset_index()[["DECLARANT_ISO", "PARTNER_ISO", criterion]]
    )
    df_comext = df_comext.sort_values(
        criterion, ascending=False
    )
    
    # Cut graph on bottom percentile
    if percentage is not None:
        SUM = df_comext[criterion].sum()
        df_comext = df_comext[
            df_comext[criterion].cumsum(skipna=False) / SUM * 100 < percentage
        ]
    
    logger.info("[TERRA] Graph table ready!")

    return df_comext

### Build graph
Build the graph and the metrics based on the filtered table

In [9]:
def build_graph(tab4graph, pos_ini, weight_flag, flow, criterion):
    
    logger.info("[TERRA] Building GRAPH...")

    # Create an empty graph
    G = nx.DiGraph()

    # Assign roles according to flow (IMPORT or EXPORT)
    if flow == 1:
        country_from = "PARTNER_ISO"
        country_to = "DECLARANT_ISO"
    else:
        country_from = "DECLARANT_ISO"
        country_to = "PARTNER_ISO"

    # Build the Graph with edges and nodes, if the Graph is weighted
    # assign the weight VALUE or QUANTITY depending on the criterion chosen to sort the market and perform the cut
    if weight_flag == True:
        weight = criterion
        WEIGHT_SUM = tab4graph[weight].sum()
        edges = [
            (i, j, w / WEIGHT_SUM)
            for i, j, w in tab4graph.loc[:, [country_from, country_to, weight]].values
        ]
    else:
        edges = [
            (i, j, 1) for i, j in tab4graph.loc[:, [country_from, country_to]].values
        ]

    # Add weigthed edges to the graph
    G.add_weighted_edges_from(edges)

    attribute = {}
    for i, j, w in edges:
        attribute[(i, j)] = {criterion: int(w * WEIGHT_SUM)}

    nx.set_edge_attributes(G, attribute)

    # Build metrics
    graph_metrics = build_metrics(G)

    # Json graph
    GG = json_graph.node_link_data(G)
    
    k_layout = 5
    pos_ini = {}
    random.seed(88)
    for node in GG["nodes"]:
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        pos_ini[node["id"]] = np.array([x, y])

    try:
        coord = nx.spring_layout(
            G, k=k_layout / math.sqrt(G.order()), pos=pos_ini, iterations=200
         )
        coord = nx.spring_layout(
            G, k=k_layout / math.sqrt(G.order()), pos=coord, iterations=50
        )  # stable solution

    except:
        return None, None, None

    # Create a dataframe with graph nodes coordinates
    df_coord = pd.DataFrame.from_dict(coord, orient="index")
    df_coord.columns = ["x", "y"]

    df = pd.DataFrame(GG["nodes"])
    df.columns = ["label"]
    df["id"] = np.arange(df.shape[0])
    df = df[["id", "label"]]
    out = pd.merge(df, df_coord, left_on="label", right_index=True)
    dict_nodes = out.T.to_dict().values()

    dfe = pd.DataFrame(GG["links"])[["source", "target", "weight", criterion]]
    res = dfe.set_index("source").join(
        out[["label", "id"]].set_index("label"), on="source", how="left"
    )
    res.columns = ["target", "source_id", "weight", criterion]
    res2 = res.set_index("target").join(
        out[["label", "id"]].set_index("label"), on="target", how="left"
    )
    res2.columns = ["weight", criterion, "from", "to"]
    res2.reset_index(drop=True, inplace=True)
    dict_edges = res2.T.to_dict().values()

    new_dict = {
        "nodes": list(dict_nodes),
        "edges": list(dict_edges),
        "metriche": graph_metrics,
    }

    JSON = json.dumps(new_dict)

    logger.info("[TERRA] GRAPH built!")

    return coord, JSON, G

In [10]:
def jsonpos2coord(jsonpos):
    logger.info("[TERRA] JSON2COORDINATES...")
    coord = {}
    for id, x, y in pd.DataFrame.from_dict(jsonpos["nodes"])[
        ["label", "x", "y"]
    ].values:
        coord[id] = np.array([x, y])
    logger.info("[TERRA] JSON2COORDINATES done!")
    return coord

In [11]:
def load_data():
    try:
        df_intra = load_intra()
        df_intra_trim = load_intra_trim()
        df_extra = load_extra()
        df_extra_trim = load_extra_trim()
    except:
        logger.info("[TERRA] Files not found!")

In [12]:
df_intra = load_intra()
df_extra = load_extra()
df_intra_trim = load_intra_trim()
df_extra_trim = load_extra_trim()

2024-01-03 12:51:51.334 INFO 17239207 - load_intra: [TERRA] Loading dataset INTRA...
2024-01-03 12:52:02.769 INFO 17239207 - load_intra: [TERRA] Dataset INTRA loaded!
2024-01-03 12:52:02.769 INFO 17239207 - load_intra: [TERRA] Dataset INTRA contains 15101732 records
2024-01-03 12:52:02.769 INFO 17239207 - load_extra: [TERRA] Loading dataset EXTRA...
2024-01-03 12:52:18.206 INFO 17239207 - load_extra: [TERRA] Dataset EXTRA loaded!
2024-01-03 12:52:18.206 INFO 17239207 - load_extra: [TERRA] Dataset EXTRA contains 22035020 records


## Flask setup

In [13]:
app = Flask(__name__)
CORS(app, resources=r'/*')

""" 
azure_exporter.add_telemetry_processor(ai_callback_function)
if is_application_insight_configured():
    middleware = FlaskMiddleware(
        app,
        exporter=azure_exporter,
        sampler=ProbabilitySampler(rate=1.0),
    ) 
"""

' \nazure_exporter.add_telemetry_processor(ai_callback_function)\nif is_application_insight_configured():\n    middleware = FlaskMiddleware(\n        app,\n        exporter=azure_exporter,\n        sampler=ProbabilitySampler(rate=1.0),\n    ) \n'

## Endpoints

### Graph extra month

In [14]:
@app.route("/graphExtraMonth", methods=["POST", "GET"])
def graphExtraMonth():
    if request.method == "POST":
        logger.info("[TERRA] Graph extra month...")

        # Currently criterio is set to "VALUE_IN_EUROS" 
        criterion = CRITERION

        # User request
        jsonRequest = dict(request.json)
        
        #Get PERCENTAGE
        percentage = int(jsonRequest["tg_perc"])
        
        #Get PERIOD
        period = int(jsonRequest["tg_period"])
        
        #Get NODES COORDINATES
        pos = jsonRequest["pos"]
        if pos == "None" or len(pos["nodes"]) == 0:
            pos = None
        else:
            # Build nodes coordinates according to previous graph
            pos = jsonpos2coord(pos)

        # 0:Unknown 1:Sea 2:Rail 3:Road 4Air 5:Post 7:Fixed Mechanism 8:Inland Waterway 9:Self Propulsion
        transports = jsonRequest["listaMezzi"]  # [0,1,2,3,4,5,7,8,9]
        
        #Get FLOW
        flow = int(jsonRequest["flow"])
        
        #Get PRODUCT
        product = str(jsonRequest["product"])
        
        #Get WEIGHT_FLAG (currently hardcoded)
        weight_flag = bool(jsonRequest["weight_flag"])
        
        # This key is set in the scenario analysis
        selectedTransportEdges = jsonRequest["selezioneMezziEdges"]
        if selectedTransportEdges == "None":
            selectedTransportEdges = None
        else:
            pass

        #Build graph table
        tab4graph = extract_graph_table(
            period,
            percentage,
            transports,
            flow,
            product,
            criterion,
            selectedTransportEdges,
            df_extra,
        )
        logger.info(f"[TERRA] Graph shape {tab4graph.shape}")
        
        # Check the size of the graph
        NUM_NODI = len(
            set(tab4graph["DECLARANT_ISO"]).union(set(tab4graph["PARTNER_ISO"]))
        )
        if NUM_NODI > MAX_NODES:
            logger.info(f"[TERRA] Graph is too wide!")
            return json.dumps({"STATUS": "05"})
        
        # Build graph
        pos, JSON, G = build_graph(tab4graph, pos, weight_flag, flow, criterion)

        if pos is None:
            if JSON is None:
                logger.info(f"[TERRA] Graph is empty!")
                return json.dumps({"STATUS": "06"})
        
        resp = Response(response=JSON, status=200, mimetype="application/json")
        logger.info("[TERRA] Graph extra month done!")
        return resp

    else:
        logger.info("[TERRA] Error in HTTP request method!")
        return str("only post")

### Graph extra trimester

In [ ]:
@app.route("/graphExtraTrim", methods=["POST", "GET"])
def graphExtraTrim():
    if request.method == "POST":
        logger.info("[TERRA] Graph extra trimester...")

        # Currently criterio is set to "VALUE_IN_EUROS" 
        criterion = CRITERION

        # User request
        jsonRequest = dict(request.json)
        
        #Get PERCENTAGE
        percentage = int(jsonRequest["tg_perc"])
        
        #Get PERIOD
        period = int(jsonRequest["tg_period"])
        
        #Get NODES COORDINATES
        pos = jsonRequest["pos"]
        if pos == "None" or len(pos["nodes"]) == 0:
            pos = None
        else:
            # Build nodes coordinates according to previous graph
            pos = jsonpos2coord(pos)

        # 0:Unknown 1:Sea 2:Rail 3:Road 4Air 5:Post 7:Fixed Mechanism 8:Inland Waterway 9:Self Propulsion
        transports = jsonRequest["listaMezzi"]  # [0,1,2,3,4,5,7,8,9]
        
        #Get FLOW
        flow = int(jsonRequest["flow"])
        
        #Get PRODUCT
        product = str(jsonRequest["product"])
        
        #Get WEIGHT_FLAG (currently hardcoded)
        weight_flag = bool(jsonRequest["weight_flag"])
        
        # This key is set in the scenario analysis
        selectedTransportEdges = jsonRequest["selezioneMezziEdges"]
        if selectedTransportEdges == "None":
            selectedTransportEdges = None
        else:
            pass

        #Build graph table
        tab4graph = extract_graph_table(
            period,
            percentage,
            transports,
            flow,
            product,
            criterion,
            selectedTransportEdges,
            df_extra_trim,
        )
        logger.info(f"[TERRA] Graph shape {tab4graph.shape}")
        
        # Check the size of the graph
        NUM_NODI = len(
            set(tab4graph["DECLARANT_ISO"]).union(set(tab4graph["PARTNER_ISO"]))
        )
        if NUM_NODI > MAX_NODES:
            logger.info(f"[TERRA] Graph is too wide!")
            return json.dumps({"STATUS": "05"})
        
        # Build graph
        pos, JSON, G = build_graph(tab4graph, pos, weight_flag, flow, criterion)

        if pos is None:
            if JSON is None:
                logger.info(f"[TERRA] Graph is empty!")
                return json.dumps({"STATUS": "06"})
        
        resp = Response(response=JSON, status=200, mimetype="application/json")
        logger.info("[TERRA] Graph extra trimester done!")
        return resp

    else:
        logger.info("[TERRA] Error in HTTP request method!")
        return str("only post")

### Graph intra month

In [15]:
@app.route('/graphIntraMonth', methods=['POST','GET'])
def graphIntraMonth():
    if request.method == 'POST':
        logger.info("[TERRA] Graph intra month...")

        # Currently criterio is set to "VALUE_IN_EUROS" 
        criterion = CRITERION

        # User request
        jsonRequest = dict(request.json)
        
        #Get PERCENTAGE
        percentage = int(jsonRequest["tg_perc"])
        
        #Get PERIOD
        period = int(jsonRequest["tg_period"])
        
        #Get NODES COORDINATES
        pos = jsonRequest["pos"]
        if pos == "None" or len(pos["nodes"]) == 0:
            pos = None
        else:
            # Build nodes coordinates according to previous graph
            pos = jsonpos2coord(pos)

        #Get FLOW
        flow = int(jsonRequest["flow"])
        
        #Get PRODUCT
        product = str(jsonRequest["product"])
        
        #Get WEIGHT_FLAG (currently hardcoded)
        weight_flag = bool(jsonRequest["weight_flag"])
        
        # This key is set in the scenario analysis
        selectedTransportEdges = jsonRequest["selezioneMezziEdges"]
        if selectedTransportEdges == "None":
            selectedTransportEdges = None
        else:
            pass

        #Build graph table (without transports)
        tab4graph = extract_graph_table(
            period,
            percentage,
            None,
            flow,
            product,
            criterion,
            selectedTransportEdges,
            df_intra,
        )
        logger.info(f"[TERRA] Graph shape {tab4graph.shape}")
        
        # Check the size of the graph
        NUM_NODI = len(
            set(tab4graph["DECLARANT_ISO"]).union(set(tab4graph["PARTNER_ISO"]))
        )
        if NUM_NODI > MAX_NODES:
            logger.info(f"[TERRA] Graph is too wide!")
            return json.dumps({"STATUS": "05"})
        
        # Build graph
        pos, JSON, G = build_graph(tab4graph, pos, weight_flag, flow, criterion)

        if pos is None:
            if JSON is None:
                logger.info(f"[TERRA] Graph is empty!")
                return json.dumps({"STATUS": "06"})
        
        resp = Response(response=JSON, status=200, mimetype="application/json")
        logger.info("[TERRA] Graph intra month done!")
        return resp
    else:
        logger.info("[TERRA] Error in HTTP request method!")
        return str("only post")

### Graph intra trimester

In [ ]:
@app.route('/graphIntraTrim', methods=['POST','GET'])
def graphIntraTrim():
    if request.method == 'POST':
        logger.info("[TERRA] Graph intra trimester...")

        # Currently criterio is set to "VALUE_IN_EUROS" 
        criterion = CRITERION

        # User request
        jsonRequest = dict(request.json)
        
        #Get PERCENTAGE
        percentage = int(jsonRequest["tg_perc"])
        
        #Get PERIOD
        period = int(jsonRequest["tg_period"])
        
        #Get NODES COORDINATES
        pos = jsonRequest["pos"]
        if pos == "None" or len(pos["nodes"]) == 0:
            pos = None
        else:
            # Build nodes coordinates according to previous graph
            pos = jsonpos2coord(pos)

        #Get FLOW
        flow = int(jsonRequest["flow"])
        
        #Get PRODUCT
        product = str(jsonRequest["product"])
        
        #Get WEIGHT_FLAG (currently hardcoded)
        weight_flag = bool(jsonRequest["weight_flag"])
        
        # This key is set in the scenario analysis
        selectedTransportEdges = jsonRequest["selezioneMezziEdges"]
        if selectedTransportEdges == "None":
            selectedTransportEdges = None
        else:
            pass

        #Build graph table (without transports)
        tab4graph = extract_graph_table(
            period,
            percentage,
            None,
            flow,
            product,
            criterion,
            selectedTransportEdges,
            df_intra_trim,
        )
        logger.info(f"[TERRA] Graph shape {tab4graph.shape}")
        
        # Check the size of the graph
        NUM_NODI = len(
            set(tab4graph["DECLARANT_ISO"]).union(set(tab4graph["PARTNER_ISO"]))
        )
        if NUM_NODI > MAX_NODES:
            logger.info(f"[TERRA] Graph is too wide!")
            return json.dumps({"STATUS": "05"})
        
        # Build graph
        pos, JSON, G = build_graph(tab4graph, pos, weight_flag, flow, criterion)

        if pos is None:
            if JSON is None:
                logger.info(f"[TERRA] Graph is empty!")
                return json.dumps({"STATUS": "06"})
        
        resp = Response(response=JSON, status=200, mimetype="application/json")
        logger.info("[TERRA] Graph intra trimester done!")
        return resp
    else:
        logger.info("[TERRA] Error in HTTP request method!")
        return str("only post")

## Start Python server

In [16]:
if __name__ == '__main__':
    IP='0.0.0.0'
    port=5500
    app.run(host=IP, port=port)


 * Serving Flask app '__main__'
 * Debug mode: off


2024-01-03 12:52:18.378 INFO _internal - _log: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5500
 * Running on http://192.168.1.198:5500
2024-01-03 12:52:18.378 INFO _internal - _log: Press CTRL+C to quit
2024-01-03 12:52:51.835 INFO _internal - _log: 127.0.0.1 - - [03/Jan/2024 12:52:51] "OPTIONS /graphExtraMonth HTTP/1.1" 200 -
2024-01-03 12:52:52.083 INFO 1721788469 - graphExtraMonth: [TERRA] Graph extra month...
2024-01-03 12:52:52.091 INFO 1457142449 - extract_graph_table: [TERRA] Preparing graph table...
2024-01-03 12:52:54.309 INFO 1457142449 - extract_graph_table: [TERRA] Graph table ready!
2024-01-03 12:52:54.309 INFO 1721788469 - graphExtraMonth: [TERRA] Graph shape (197, 3)
2024-01-03 12:52:54.315 INFO 3536347606 - build_graph: [TERRA] Building GRAPH...
2024-01-03 12:52:54.315 INFO 1748329586 - build_metrics: [TERRA] Calculating graph 